In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
zip_path = '/content/drive/MyDrive/ Hanifa Bunayya-Computer Vision Project/HAM10000.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Selesai ekstrak!")

Mounted at /content/drive
Selesai ekstrak!


In [ ]:
import pandas as pd
import shutil
import os

# Path metadata dan folder hasil ekstrak
metadata_path = '/content/drive/MyDrive/ Hanifa Bunayya-Computer Vision Project/HAM10000_metadata.csv'
image_dir = '/content/HAM10000'  # hasil ekstrak ZIP
output_dir = '/content/bcc_binary'

bcc_dir = os.path.join(output_dir, 'bcc')
non_bcc_dir = os.path.join(output_dir, 'non_bcc')

os.makedirs(bcc_dir, exist_ok=True)
os.makedirs(non_bcc_dir, exist_ok=True)

df = pd.read_csv(metadata_path)

# Salin gambar sesuai label
for _, row in df.iterrows():
    img_id = row['image_id']
    label = row['dx']
    src = os.path.join(image_dir, img_id + '.jpg')

    if not os.path.exists(src):
        continue

    dst_dir = bcc_dir if label == 'bcc' else non_bcc_dir
    shutil.copy(src, os.path.join(dst_dir, img_id + '.jpg'))

print("✅ Selesai memisahkan gambar ke bcc dan non_bcc.")

✅ Selesai memisahkan gambar ke bcc dan non_bcc.


In [ ]:
import os
import shutil
import random

source_base = '/content/bcc_binary'
split_base = '/content/bcc_binary_split'

# Buat folder struktur train/val/test untuk kedua kelas
for split in ['train', 'val', 'test']:
    for cls in ['bcc', 'non_bcc']:
        os.makedirs(os.path.join(split_base, split, cls), exist_ok=True)

# Fungsi untuk split dan copy file
def split_and_copy(source_dir, class_name):
    files = os.listdir(source_dir)
    random.shuffle(files)

    n = len(files)
    train_split = int(0.7 * n)
    val_split = int(0.85 * n)

    train_files = files[:train_split]
    val_files = files[train_split:val_split]
    test_files = files[val_split:]

    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(split_base, 'train', class_name, f))
    for f in val_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(split_base, 'val', class_name, f))
    for f in test_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(split_base, 'test', class_name, f))

# Lakukan split untuk kedua kelas
split_and_copy(os.path.join(source_base, 'bcc'), 'bcc')
split_and_copy(os.path.join(source_base, 'non_bcc'), 'non_bcc')

print("✅ Dataset berhasil di-split menjadi train, val, dan test untuk kedua kelas.")


✅ Dataset berhasil di-split menjadi train, val, dan test untuk kedua kelas.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    '/content/bcc_binary_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_gen = datagen.flow_from_directory(
    '/content/bcc_binary_split/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 1 neuron = prediksi kemiripan dengan BCC
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=10)


Found 359 images belonging to 1 classes.
Found 77 images belonging to 1 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 47s 4s/step - accuracy: 0.7064 - loss: 0.2170 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 1.00

In [ ]:
model.save('/content/drive/MyDrive/ Hanifa Bunayya-Computer Vision Project/bcc_vs_nonbcc_model.h5')

In [ ]:
test_gen = datagen.flow_from_directory(
    '/content/bcc_binary_split/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_gen)
print(f"Akurasi data test: {test_acc:.4f}, Loss: {test_loss:.4f}")


Found 78 images belonging to 1 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 767ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Akurasi data test: 1.0000, Loss: 0.0000


In [ ]:
!pip install flask-ngrok


In [ ]:
!unzip -o ngrok.zip
!chmod +x ngrok


unzip:  cannot find or open ngrok.zip, ngrok.zip.zip or ngrok.zip.ZIP.
chmod: cannot access 'ngrok': No such file or directory


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Ganti dengan authtoken kamu
ngrok.set_auth_token("2zKFWFvfJG19UYHhG4mRk231Rh0_4DXVtZYP3heL8hSiFbLXt")


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2zKFWFvfJG19UYHhG4mRk231Rh0_4DXVtZYP3heL8hSiFbLXt")
public_url = ngrok.connect(5000)
print(public_url)

NgrokTunnel: "https://6958-34-75-60-137.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import tensorflow as tf
import numpy as np
from PIL import Image
import os

app = Flask(__name__)
CORS(app)

MODEL_PATH = 'bcc_vs_nonbcc_model.h5'

if not os.path.exists(MODEL_PATH):
    print(f"Error: Model file not found at {MODEL_PATH}")
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

def preprocess(img):
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    return np.expand_dims(img, 0)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({"error": "Model not loaded. Please check server logs."}), 500

    if 'image' not in request.files:
        return jsonify({"error": "No image file provided"}), 400

    try:
        file = request.files['image']
        if file.filename == '':
            return jsonify({"error": "No selected file"}), 400

        img = Image.open(file.stream).convert('RGB')

        input_tensor = preprocess(img)
        pred = model.predict(input_tensor)[0][0]
        print("Prediksi mentah (confidence):", float(pred))

        result = "Basal Cell Carcinoma" if pred >= 0.5 else "Non-Basal Cell Carcinoma"

        return jsonify({
            "result": result,
            "confidence": float(pred)
        })
    except Exception as e:
        print(f"Error during prediction: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5050, debug=True)

Writing app.py


In [ ]:
from google.colab import files
files.download('app.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>